# Unemployment in India and Credit Card Spendings

We have data from two context:
- Unemployment in India
- Credit Card Spendings

In [4]:
import pandas as pd
from ydata_profiling import ProfileReport

c:\Users\ismae\OneDrive\Cursos\AI ML DS CMU\1. Foundation of Data Science\cmu-ds-foundation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the data

In [5]:
df_spendings = pd.read_csv(r'.\kaggle\input\Credit card transactions - India - Simple.csv')
df_unemployment = pd.read_csv(r'.\kaggle\input\Unemployment in India.csv')

## Profiling the data

In [6]:
# Unployment data
profile_unemployment = ProfileReport(df_unemployment, title='Unemployment in India', explorative=True)

# Spendings data
profile_spendings = ProfileReport(df_spendings, title='Credit card transactions - India - Simple', explorative=True)

In [8]:
# Save the reports
profile_unemployment.to_file(r'.\profiles\unemployment_data_profile.html')
profile_spendings.to_file(r'.\profiles\spendings_data_profile.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 124.97it/s]


## Exploring the data

In [12]:
df_unemployment.head()

,Region,Date,Frequency,Estimated Unemployment Rate (%),Estimated Employed,Estimated Labour Participation Rate (%),Area
0,Andhra Pradesh,31-05-2019,Monthly,3.65,11999139.0,43.24,Rural
1,Andhra Pradesh,30-06-2019,Monthly,3.05,11755881.0,42.05,Rural
2,Andhra Pradesh,31-07-2019,Monthly,3.75,12086707.0,43.50,Rural
3,Andhra Pradesh,31-08-2019,Monthly,3.32,12285693.0,43.97,Rural
4,Andhra Pradesh,30-09-2019,Monthly,5.17,12256762.0,44.68,Rural


In [15]:
df_spendings.head()

,df_index,City,Date,Card Type,Exp Type,Gender,Amount
0,0,"Delhi, India",29-Oct-14,Gold,Bills,F,82475
1,1,"Greater Mumbai, India",22-Aug-14,Platinum,Bills,F,32555
2,2,"Bengaluru, India",27-Aug-14,Silver,Bills,F,101738
3,3,"Greater Mumbai, India",12-Apr-14,Signature,Bills,F,123424
4,4,"Bengaluru, India",5-May-15,Gold,Bills,F,171574


#### Integrity constraints 

**Unemployment data**:  

- We should have unique month-year for each row  
- Percentage should be between 0 and 100  
- There should be only Urban and Rural as values for Area

**Spendings data**:

- Amount should be positive and greater than 0 and not null
- Date should be filled
- City should be filled to allow integration with unemployment data